In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
from data import Team, TEAM_ABBREVIATIONS_TO_TEAM, TEAM_TO_TEAM_ABBREVIATION
from multiprocessing import Pool
from pathlib import Path
import os


In [3]:
df_bs = pd.read_csv(".data/raw_data/box_scores/2010_box_scores.csv").drop(columns=['Unnamed: 0'])
df_schedules = pd.read_csv(".data/raw_data/nba_schedules/2010_schedule_matches.csv").drop(columns=['Unnamed: 0'])

In [4]:
# numeric_columns = df_bs.columns[~df_bs.columns.isin(['PLAYER', 'GAME_DATE', 'TEAM', 'MP'])].values.tolist()
#
# for column in numeric_columns:
#     df_bs.loc[:, column] = pd.to_numeric(df_bs.loc[:, column])
#
# # df_bs.loc[:, numeric_columns] = df_bs.loc[:, numeric_columns].map(pd.to_numeric)

In [5]:
df_schedules

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS
0,2009-10-27,Boston Celtics,95,Cleveland Cavaliers,89
1,2009-10-27,Washington Wizards,102,Dallas Mavericks,91
2,2009-10-27,Houston Rockets,87,Portland Trail Blazers,96
3,2009-10-27,Los Angeles Clippers,92,Los Angeles Lakers,99
4,2009-10-28,Indiana Pacers,109,Atlanta Hawks,120
...,...,...,...,...,...
1225,2010-04-14,New York Knicks,113,Toronto Raptors,131
1226,2010-04-14,Indiana Pacers,97,Washington Wizards,98
1227,2010-04-14,Los Angeles Lakers,91,Los Angeles Clippers,107
1228,2010-04-14,Golden State Warriors,122,Portland Trail Blazers,116


In [8]:
def encode_dnp_dnd(df):
    return df.assign(DNP_DND=lambda x: (x.MP=='Did Not Play') | (x.MP=='Did Not Dress')).astype({'DNP_DND': 'int64'})

def replace_dnp(df):
    return df.replace('Did Not Play', method='backfill')

def reindex_players(df, schedule):
    player_col = 'PLAYER'
    
    other_cols = [col for col in df.columns[df.columns != player_col]]
    
    player = df.loc[:, player_col].iloc[0]
    print(df[df.index.value_counts() > 1])
    
#     df[(df.index.value_counts() > 1) & (df.)]
    
#     df = df.drop_duplicates()
    df = df.reindex(schedule, fill_value=player)

    return df.loc[:, other_cols].replace(to_replace=player, value='Did Not Dress')

In [9]:
years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']


for team in Team:
    Path(f".data/not_raw_data/{team.value}/").mkdir(parents=True, exist_ok=True)

    for year in years:
        Path(f".data/not_raw_data/{team.value}/{year}/").mkdir(parents=True, exist_ok=True)

        df_schedules = pd.read_csv(f".data/raw_data/nba_schedules/{year}_schedule_matches.csv").drop(columns=['Unnamed: 0'])
        
        df_bs = pd.read_csv(f".data/raw_data/box_scores/{year}_box_scores.csv").drop(columns=['Unnamed: 0'])
        df_bs = df_bs.loc[TEAM_ABBREVIATIONS_TO_TEAM[df_bs.TEAM] == team.value]
        
        df_team = df_schedules[(df_schedules.HOME.str.upper() == team.value) | (df_schedules.VISITOR.str.upper() == team.value)]

        df_team.loc[:, "HOME_ABBR"] = df_team.HOME.apply(lambda x: TEAM_TO_TEAM_ABBREVIATION[Team(x.upper())])
        df_team.loc[:, "VISITOR_ABBR"] = df_team.VISITOR.apply(lambda x: TEAM_TO_TEAM_ABBREVIATION[Team(x.upper())])

        df_bs_team = df_bs[df_bs.TEAM == TEAM_TO_TEAM_ABBREVIATION[team]].sort_values(['PLAYER', 'GAME_DATE'])#.set_index('GAME_DATE')

        df_bs_team = df_bs_team[df_bs_team.PLAYER != 'Team Totals']
        df_bs_team_totals = df_bs_team[df_bs_team.PLAYER == 'Team Totals']

        team_schedule_ls = df_team.DATE.values.tolist()

        df_bs_team_reidx = df_bs_team.groupby(['PLAYER','TEAM']).apply(reindex_players, team_schedule_ls).reset_index().set_index('GAME_DATE')
        df_bs_team_reidx = df_bs_team_reidx.groupby(['PLAYER']).apply(encode_dnp_dnd)

        df_bs_team_totals.to_csv(f".data/not_raw_data/{team.value}/{year}/{team.value.lower()}_{year}_team_totals.csv")
        df_bs_team_reidx.to_csv(f".data/not_raw_data/{team.value}/{year}/{team.value.lower()}_{year}_box_score.csv")
        df_team.to_csv(f".data/not_raw_data/{team.value}/{year}/{team.value.lower()}_{year}_schedule.csv")


TypeError: unhashable type: 'Series'

In [43]:
for team in Team:
    # Path(f".data/not_raw_data/{team.value}/").mkdir(parents=True, exist_ok=True)
    df_team = df_schedules[(df_schedules.HOME.str.upper() == team.value) | (df_schedules.VISITOR.str.upper() == team.value)]

    df_team.loc[:, "HOME_ABBR"] = df_team.HOME.apply(lambda x: TEAM_TO_TEAM_ABBREVIATION[Team(x.upper())])
    df_team.loc[:, "VISITOR_ABBR"] = df_team.VISITOR.apply(lambda x: TEAM_TO_TEAM_ABBREVIATION[Team(x.upper())])

    team_schedule_ls = df_team.DATE.values.tolist()

    df_bs_team = df_bs[df_bs.TEAM == TEAM_TO_TEAM_ABBREVIATION[team]].sort_values(['PLAYER', 'GAME_DATE']).set_index('GAME_DATE')

    player_game_attendance = df_bs_team.groupby('PLAYER')['MP'].count() / len(team_schedule_ls)
    valid_player_ls = player_game_attendance[player_game_attendance > 0.35].index.values.tolist()

    df_bs_players = df_bs_team[df_bs_team.PLAYER != 'Team Totals']
    df_bs_players = df_bs_players[df_bs_players.PLAYER.isin(valid_player_ls)]
    df_bs_team_totals = df_bs_team[df_bs_team.PLAYER == 'Team Totals']

    print(player_game_attendance, valid_player_ls, df_bs_players)

    df_bs_players_reidx = df_bs_players.groupby(['PLAYER']).apply(reindex_players, team_schedule_ls).reset_index().set_index('GAME_DATE')
    df_bs_players_reidx = df_bs_players_reidx.groupby(['PLAYER']).apply(encode_dnp_dnd)
    # df_bs_team_reidx.to_csv("")

PLAYER
Al Horford        0.353659
Alan Anderson     0.024390
Alexis Ajinça     0.012195
Arron Afflalo     0.012195
Austin Daye       0.024390
                    ...   
Tony Mitchell     0.036585
Travis Outlaw     0.012195
Tyshawn Taylor    0.012195
Udonis Haslem     0.024390
Will Bynum        0.012195
Name: MP, Length: 88, dtype: float64 ['Al Horford', 'Cartier Martin', 'DeMarre Carroll', 'Elton Brand', 'Jeff Teague', 'Kyle Korver', 'Lou Williams', 'Mike Scott', 'Paul Millsap', 'Shelvin Mack', 'Team Totals']                   PLAYER            MP            FG           FGA  \
GAME_DATE                                                            
2013-10-30    Al Horford         31:59             5            12   
2013-11-01    Al Horford         35:13             9            14   
2013-11-03    Al Horford         33:37             5            12   
2013-11-05    Al Horford         35:16            11            19   
2013-11-07    Al Horford         35:07            10            2

C:\Users\rivverg\AppData\Local\Temp\ipykernel_888\362028662.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_team.loc[:, "HOME_ABBR"] = df_team.HOME.apply(lambda x: TEAM_TO_TEAM_ABBREVIATION[Team(x.upper())])
C:\Users\rivverg\AppData\Local\Temp\ipykernel_888\362028662.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_team.loc[:, "VISITOR_ABBR"] = df_team.VISITOR.apply(lambda x: TEAM_TO_TEAM_ABBREVIATION[Team(x.upper())])
C:\Users\rivverg\AppData\Local\Temp\ipykernel_888\1765025538.py:13: FutureWa

ValueError: cannot reindex on an axis with duplicate labels

2014-02-11, 2014-01-04, 2013-11-30

In [34]:
df_bs_team_reidx.index.value_counts()

2012-11-02    18
2013-03-09    18
2013-03-06    18
2013-03-04    18
2013-03-03    18
              ..
2012-12-28    18
2012-12-26    18
2012-12-22    18
2012-12-21    18
2013-04-17    18
Name: GAME_DATE, Length: 82, dtype: int64

In [10]:
numeric_columns = df_bs.columns[~df_bs.columns.isin(['PLAYER', 'GAME_DATE', 'TEAM', 'MP'])].values.tolist()

def dnp_dnd_encoding(value):
    pass
    # if (value == "Did Not Play") | (value == "Did Not Dress"):
    #     return 1
    # else:
    #     return 0

df_bs_team.loc[:, numeric_columns].apply(lambda x: dnp_dnd_encoding(x))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [66]:
def replace_dnp(df):
    return df.sort_values(['PLAYER', 'GAME_DATE']).replace('Did Not Play', method='backfill')

In [67]:
df_bs_team.groupby(['PLAYER']).apply(replace_dnp)

,Unnamed: 0,PLAYER,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,GAME_DATE,TEAM
104,0,Joe Johnson,39:23,10,21,.476,2,5,.400,3,...,0,6,4,0,4,2,25,+17,2009-10-28,ATL
105,1,Josh Smith,35:12,7,10,.700,0,0,NaN,4,...,3,8,5,2,1,4,18,+23,2009-10-28,ATL
106,2,Al Horford,33:31,11,17,.647,0,0,NaN,2,...,16,4,0,2,1,3,24,+15,2009-10-28,ATL
107,3,Marvin Williams,31:14,4,8,.500,1,1,1.000,5,...,4,0,2,2,2,4,14,+12,2009-10-28,ATL
108,4,Mike Bibby,30:43,4,7,.571,2,3,.667,0,...,4,5,3,0,2,1,10,+16,2009-10-28,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31617,8,Mike Bibby,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,2010-04-14,ATL
31618,9,Al Horford,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,2010-04-14,ATL
31619,10,Joe Johnson,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,...,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,Did Not Dress,2010-04-14,ATL
31620,11,Josh Smith,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,2010-04-14,ATL


In [6]:
df_bs_players

NameError: name 'df_bs_players' is not defined

In [5]:
df_bs_players

NameError: name 'df_bs_players' is not defined

In [39]:
df_schedules

,Unnamed: 0,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS
0,0,2009-10-27,Boston Celtics,95,Cleveland Cavaliers,89
1,1,2009-10-27,Washington Wizards,102,Dallas Mavericks,91
2,2,2009-10-27,Houston Rockets,87,Portland Trail Blazers,96
3,3,2009-10-27,Los Angeles Clippers,92,Los Angeles Lakers,99
4,4,2009-10-28,Indiana Pacers,109,Atlanta Hawks,120
...,...,...,...,...,...,...
1225,1225,2010-04-14,New York Knicks,113,Toronto Raptors,131
1226,1226,2010-04-14,Indiana Pacers,97,Washington Wizards,98
1227,1227,2010-04-14,Los Angeles Lakers,91,Los Angeles Clippers,107
1228,1228,2010-04-14,Golden State Warriors,122,Portland Trail Blazers,116


In [46]:
df_bs

,PLAYER,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,GAME_DATE,TEAM
0,LeBron James,44:55,12,22,.545,4,9,.444,10,13,...,4,8,2,4,5,4,38,0,2009-10-27,CLE
1,Anthony Parker,40:10,3,9,.333,2,3,.667,2,2,...,3,4,1,2,3,0,10,+3,2009-10-27,CLE
2,Mo Williams,35:02,3,8,.375,0,3,.000,6,6,...,0,3,1,0,2,4,12,-10,2009-10-27,CLE
3,Anderson Varejão,32:50,3,9,.333,0,0,NaN,3,4,...,7,1,2,1,0,5,9,-6,2009-10-27,CLE
4,Ben Wallace,28:48,5,11,.455,0,0,NaN,0,2,...,10,1,0,1,1,2,10,+2,2009-10-27,CLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31968,Deron Williams,20:05,4,7,.571,1,3,.333,3,6,...,4,3,2,0,2,3,12,+4,2010-04-14,PHO
31969,Lou Amundson,19:08,0,1,.000,0,0,NaN,4,6,...,5,0,0,4,0,5,4,+4,2010-04-14,PHO
31970,Earl Clark,3:02,0,1,.000,0,0,NaN,2,2,...,1,0,0,0,1,0,2,-2,2010-04-14,PHO
31971,Dwayne Jones,1:18,0,0,NaN,0,0,NaN,0,0,...,0,0,0,0,0,1,0,-1,2010-04-14,PHO
